In [1]:
import os
import pandas as pd
import itertools
import numpy as np
from sklearn import preprocessing as prepro

In [2]:
os.listdir("peru-data")

['sales.csv',
 'clients_attributes.csv',
 'executed_promos.csv',
 'active_promos.csv']

In [3]:
datas = ["peru-data/"+str(i) for i in os.listdir('peru-data')]

In [4]:
clientAtrDf = pd.read_csv(datas[1],encoding="latin1")
executedPromosDf = pd.read_csv(datas[2],encoding="latin1")
active_promosDf  = pd.read_csv(datas[3],encoding="latin1")
salesDf = pd.read_csv(datas[0],encoding="latin1")

In [5]:
allClients = pd.unique(clientAtrDf["Cliente"])
allBrands  = pd.unique(active_promosDf["Marca"])
allCupos   = pd.unique(active_promosDf["Cupo"])

In [6]:
b1 = pd.DataFrame(allClients,columns=["cliente"])
b1["marca"] =allBrands[0]
b2= pd.DataFrame(allClients,columns=["cliente"])
b2["marca"] =allBrands[1]
b3 = pd.DataFrame(allClients,columns=["cliente"])
b3["marca"] =allBrands[2]

temp = pd.concat((b1,b2),axis=0,ignore_index=True)
c1 = pd.concat((temp,b3),axis=0,ignore_index=True)
c2 = pd.concat((temp,b3),axis=0,ignore_index=True)
c3 = pd.concat((temp,b3),axis=0,ignore_index=True)

c1["cupo"]=allCupos[0]
c2["cupo"]=allCupos[1]
c3["cupo"]=allCupos[2]

temp = pd.concat((c1,c2),axis=0,ignore_index=True)
keyColumnsDf = pd.concat((temp,c3),axis=0,ignore_index=True)

In [7]:
#keyColumnsDf.head(5).to_csv("keyColumns.csv")

In [8]:
clientAtrDf[clientAtrDf["Cliente"]==4000]

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
3188,4000,2010-04-27,3,10,18,2,1,0


In [9]:
salesDf["token"] = salesDf["Cliente"].map(lambda x: str(x)) + salesDf["Marca"].map(lambda x: str(x))+salesDf["Cupo"].map(lambda x: str(x))


In [10]:
salesDf[salesDf["token"]=="104016.0"]

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,token
3375,10,2019,7,2,1.0,40,16.0,12909.04,0.06,-6253.02,104016.0
3376,10,2019,8,2,1.0,40,16.0,18385.05,0.09,-10363.05,104016.0
14690,10,2019,6,2,1.0,40,16.0,45181.63,0.21,-21885.57,104016.0


In [11]:
executedPromosDf["token"] = executedPromosDf["Cliente"].map(lambda x: str(x)) + executedPromosDf["Marca"].map(lambda x: str(x))+executedPromosDf["Cupo"].map(lambda x: str(x))

In [12]:
executedPromosDf[executedPromosDf["token"]=="104016"]


,CodigoDC,Cliente,Marca,Cupo,token
4045,617782,10,40,16,104016
23094,641612,10,40,16,104016


In [13]:
testDf = salesDf[salesDf["token"]=="104016.0"]

In [14]:
temp1=testDf[["Nr","Hl","Dcto"]]

In [15]:
temp1

,Nr,Hl,Dcto
3375,12909.04,0.06,-6253.02
3376,18385.05,0.09,-10363.05
14690,45181.63,0.21,-21885.57


In [16]:
temp2 = prepro.normalize(temp1,norm="l2")

In [17]:
temp3 = pd.DataFrame(temp2,columns=["Nr","Hl","Dcto"])

In [18]:
temp3

,Nr,Hl,Dcto
0,0.899976,0.000004,-0.435940
1,0.871141,0.000004,-0.491033
2,0.899976,0.000004,-0.435940


In [19]:
Nr = np.floor(temp3["Nr"]*(10/temp3["Nr"].max()))
Hl = np.floor(temp3["Hl"]*(10/temp3["Hl"].max()))
Dcto = np.floor(temp3["Dcto"]*(10/temp3["Dcto"].max()))

In [20]:
Nr.to_numpy().reshape(3,1)
Hl.to_numpy().reshape(3,1)
Dcto.to_numpy().reshape(3,1)

array([[10.],
       [11.],
       [10.]])

In [21]:
Dcto.to_numpy().T

array([10., 11., 10.])

In [22]:
import pingouin as pg
data = pg.read_dataset('cronbach_alpha')
#data[data["Items"]=="Q1"]
pg.cronbach_alpha(data=data, items='Items', scores='Scores',
                  subject='Subj')

(0.5917188485995826, array([0.195, 0.84 ]))

In [23]:
data[data["Subj"]==0]

,Subj,Items,Scores
0,0,Q1,3
15,0,Q2,2
30,0,Q3,4
45,0,Q4,1
60,0,Q5,4
75,0,Q6,5
90,0,Q7,1
105,0,Q8,4
120,0,Q9,3
135,0,Q10,2
